In [1]:
import torch
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
from Bio import Medline
import ollama
from fastapi import FastAPI
import uvicorn

if torch.cuda.is_available():
  device = "cuda:0"
else:
  device = "cpu"
device

records = {}
missed = 0



/home/l0gically/miniconda3/envs/HDHW/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [11]:
"""
Add the whole path if pubmed_data is not detectet
e.g. /home/paperspace/QA-INLPT-WS2023/chat-ui-rag/src/lib/server/rag/pinecone/pubmed_data
"""
PUBMED_PATH = "/home/l0gically/QA-INLPT-WS2023/chat-ui-rag/src/lib/server/rag/pinecone/pubmed_data"
with open(PUBMED_PATH) as stream:
    for article in Medline.parse(stream):

        if not "PMID" in article:
            missed += 1
            continue

        if not "TI" in article:
            missed += 1
            continue

        if not "FAU" in article:
            missed += 1
            continue

        if not "DP" in article:
            missed += 1
            continue

        if not "AB" in article:
            missed += 1
            continue

        if not "SO" in article:
            missed += 1
            continue
        
        records[article["PMID"]] = article

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

index_name = 'pubmed'
pinecone = Pinecone(api_key="7377f10c-e728-4408-b2a6-3ac28daf468f")
index = pinecone.Index(index_name)

def retrieve_documents(question):
    results = index.query(vector=model.encode([question])[0].tolist(), top_k=1)
    return [(res["id"],res["score"]) for res in results.matches]


In [14]:
def generate(question: str):
    documents = retrieve_documents(question)
    #a,b = documents[0]
    #print(records[a])
    return {"answer": [f"DOCUMENT-ID: {records[id]['PMID']}\n FULL-AUTHOR: {records[id]['FAU']}\n PUBLICATION-DATE: {records[id]['DP']}\n TEXT: {records[id]['AB']}\n SCORE: {round(score,2)} \n DOCUMENT-TITLE: {records[id]['TI']}\n SOURCE: {records[id]['SO'].split("doi:")}" for id,score in documents]}



In [16]:
generate("How do I make my kids smarter?")

{'answer': ["DOCUMENT-ID: 26172250\n FULL-AUTHOR: ['Protzko, John', 'Aronson, Joshua', 'Blair, Clancy']\n PUBLICATION-DATE: 2013 Jan\n TEXT: Can interventions meaningfully increase intelligence? If so, how? The Database of Raising Intelligence is a continuously updated compendium of randomized controlled trials that were designed to increase intelligence. In this article, the authors examine nearly every available intervention involving children from birth to kindergarten, using meta-analytic procedures when more than 3 studies tested similar methods and reviewing interventions when too few were available for meta-analysis. This yielded 4 meta-analyses on the effects of dietary supplementation to pregnant mothers and neonates, early educational interventions, interactive reading, and sending a child to preschool. All 4 meta-analyses yielded significant results: Supplementing infants with long-chain polyunsaturated fatty acids, enrolling children in early educational interventions, read